### Remove:
* Null values, new_line("\n"), hashtags("#*"), emojis, other characters
### Replace:
* ['ሐ', 'ሑ', 'ሒ', 'ሓ', 'ሔ', 'ሖ'] with ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']
* ['ኀ', 'ኁ', 'ኂ', 'ኃ', 'ኄ', 'ኅ', 'ኆ'] with ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']
* ['ሠ', 'ሡ', 'ሢ', 'ሣ', 'ሤ', 'ሦ', 'ሦ', 'ሧ'] with ['ሰ, 'ሱ', 'ሲ', 'ሳ', 'ሴ', 'ስ', 'ሶ', 'ሷ']
* ['ዐ', 'ዑ', 'ዒ', 'ዓ', 'ዔ', 'ዕ', 'ዖ'] with ['አ', 'ኡ', 'ኢ', 'ኣ', 'ኤ', 'እ', 'ኦ']
* ['ጸ', 'ጹ', 'ጺ', 'ጻ', 'ጼ', 'ጽ', 'ጾ'] with ['ፀ', 'ፁ', 'ፂ', 'ፃ', 'ፄ', 'ፅ', 'ፆ']


#### Import modules

In [1]:
import sys, os, json, re, zipfile, csv
import pandas as pd
# sys.path.append(os.path.abspath(os.path.join('../scripts')))

In [ ]:
class Util():
    def __init__(self) -> None:
        self.emoji_pattern = re.compile("["
                                        u"\U0001F600-\U0001F64F"  # Emoticons
                                        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols and Pictographs
                                        u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
                                        u"\U0001F700-\U0001F77F"  # Alchemical Symbols
                                        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                                        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                                        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                                        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                                        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                                        u"\u2600-\u26FF"  # Miscellaneous Symbols
                                        u"\u2700-\u27BF"  # Dingbats
                                        u"\u2B50"  # Star
                                        u"\U0001F1E6-\U0001F1FF"  # Country Flags
                                        "]+", flags=re.UNICODE)
        self.symbols = re.compile("["
                                  "\""
                                  "\“"
                                  "\""
                                  "\'"
                                  "\-"
                                  "\*"
                                  "\•"
                                  "\ℹ"
                                  "\﻿"
                                  "\_"
                                  "]+")
        self.url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        self.mention_pattern = r'@(\w+)'

    def read_file(self, file_path: str) -> dict:
        # Open the file for reading
        with open(file_path, 'r') as file:
            # Load the JSON data from the file
            data = json.load(file)
            return data

    def write_file(self, file_path: str, data: dict) -> None:
        # Open the file for writing
        with open(file_path, 'w') as file:
            # Dump the JSON data to the file
            json.dump(data, file, indent=2)

    def parse_text(self, text: any) -> str:
        if isinstance(text, str):
            return text
        elif isinstance(text, list):
            contents = []
            for item in text:
                if isinstance(item, str):
                    contents.append(item)
                elif isinstance(item, dict):
                    contents.append(item['text'])
            return "".join(contents)
        else:
            return ""

    def parse_messages(self, messages: list) -> dict:
        parsed_messages = {
            'id': [],
            'text': [],
            'date': []
        }
        for message in messages:
            if message['type'] != 'message' or len(message['text']) == 0:
                continue
            parsed_messages['id'].append(message['id'])
            message_content = self.parse_text(message['text'])
            parsed_messages['text'].append(message_content)
            parsed_messages['date'].append(message['date'])
        return parsed_messages

    def extract_hashtags(self, text: str) -> list:
        return [word for word in text.split() if word.startswith('#')]

    def extract_emojis(self, text):
        return ''.join(self.emoji_pattern.findall(text))

    def remove_emojis(self, text):
        return self.emoji_pattern.sub('', text)

    def extract_symbols(self, text):
        return ''.join(self.symbols.findall(text))

    def remove_symbols(self, text):
        return self.symbols.sub(' ', text)

    def extract_urls(self, text):
        return re.findall(self.url_pattern, text)

    def extract_mentions(self, text):
        return re.findall(self.mention_pattern, text)
    
    def extract_fields(self, message):
        """
        Extracts relevant fields from the message.
        Returns a tuple containing (channel_id, text, date, labels).
        """
        text = ' '.join(item['text'] for item in message['text_entities'] if item['type'] in 'plain')
        date = message['date']
        labels = "LABEL"  # Replace 'your_label' with the actual label(s) relevant to your use case
        return text, date, labels

    def process_json_file(self, json_file, csv_writer):
        """
        Processes a JSON file, extracts relevant fields, and writes to CSV.
        """
        data = json.load(json_file)

        channel_id = data['id']
        for message in data['messages']:
            text, date, labels = self.extract_fields(message)
            csv_writer.writerow([channel_id, text, date, labels])

    def process_zip(self, zip_file_path, output_csv_path):
        """
        Processes a zip file, extracts data from JSON files, and writes to a CSV file.
        """
        with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
            with open(output_csv_path, 'w', newline='', encoding='utf-8') as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow(['id', 'text', 'date', 'label'])

                for file_info in zip_file.infolist():
                    with zip_file.open(file_info.filename) as json_file:
                        print(json_file)
                        self.process_json_file(json_file, csv_writer)
                        
    def file_reader(self, path: str, ) -> str:
        fname = os.path.join(path)
        with open(fname, 'r') as f:
            system_message = f.read()
        return system_message


In [13]:
util = Util()

## Parsing the zip data file

We will create a csv file with the following files from all json files found in the zip file containing telegram channel infos in json format. The parsed file will have the following cols.


- id: Telegram Channel ID
- text: message content
- date: message broadcast datetime.
- label (s): one or more data labels relevant to your supervised training 

In [14]:
# zip_file_path = os.path.join("../data/raw/raw.zip") 
# output_csv_path = os.path.join("../data/parsed/parsed.csv") 
zip_file_path = "../data/raw/raw.zip"
output_csv_path = "../data/parsed/parsed.csv"

util.process_zip(zip_file_path, output_csv_path)
print("Parsing completed. Output saved to", output_csv_path)

<zipfile.ZipExtFile name='4-3-3 FAST SPORT™.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='90 ደቂቃ ስፖርት™.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='DID U KNOW️⁉️.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='DREAM APP™.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='DREAM SPORT ™.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='ETHIO ARSENAL.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='ETHIO-MEREJA®.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='Ethio University News®.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='History 📚.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='Manchester United Fans™.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='QUBEE ACADEMY.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='Sheger Press️️.json' mode='r' compress_type=deflate>
<zipfile.ZipExtFile name='THE GOAT LM♾ 🐐.json' m

## Cleaning the parsed data

We will cleaning the text column of the parsed file. We will be cleaning the following things:

1. **Clean Null Values:** Remove null or empty values in the 'text' column.
   
2. **Clean New Lines:** Remove extra line breaks or new lines from the 'text' column.

3. **Remove Hashtags:** Remove hashtags from the 'text' column.

4. **Remove Emojis:** Remove emojis from the 'text' column.

5. **Remove Symbols:** Remove special symbols from the 'text' column.

6. **Remove Links:** Remove hyperlinks or URLs from the 'text' column.

7. **Remove Mentions:** Remove mentions or references (e.g., @username) from the 'text' column.

8. **Remove Extra Spaces:** Remove extra spaces, multiple spaces, or leading/trailing spaces from the 'text' column.

9. **Remove Non-Amharic Characters:** Remove characters that are not part of the Amharic script from the 'text' column.



In [19]:
# Specify the paths
parsed_csv_path = "../data/parsed/parsed.csv"
output_cleaned_csv_path = "../data/parsed/cleaned_parsed.csv"

# Read the parsed CSV into a DataFrame
df = pd.read_csv(parsed_csv_path)

### 1. Clean Null Values

In [20]:
# Apply method to clean null or empty values in the 'text' column
df['text'] = df['text'].apply(util.parse_text)

### 2. Clean New Lines

In [21]:
# Remove newlines from the 'text' column
df['text'] = df['text'].replace('\n', ' ', regex=True)

### 3. Remove Hashtags

In [22]:
# Remove hashtags from the 'text' column
df['text'] = df['text'].str.replace(r'\#\w+', '', regex=True)


### 4. Remove Emojis

In [23]:
# Apply method to remove emojis from the 'text' column
df['text'] = df['text'].apply(util.remove_emojis)

### 5. Remove Symbols 

In [24]:
# Apply method to remove special symbols from the 'text' column
df['text'] = df['text'].apply(util.remove_symbols)

### 6. Remove Links 

In [31]:
# Apply method to remove hyperlinks or URLs from the 'text' column
df['text'] = df['text'].str.replace(util.url_pattern, '', regex=True).str.strip()

### 7. Remove Mentions 

In [29]:
df['text'] = df['text'].str.replace(util.mention_pattern, '', regex=True).str.strip()

### 8. Remove Extra Spaces 

In [32]:
# Apply method to remove extra spaces, multiple spaces, or leading/trailing spaces from the 'text' column
df['text'] = df['text'].str.replace('\s+', ' ', regex=True).str.strip()
df['text'] = df['text'].replace(r'!+', '!', regex=True)
df['text'] = df['text'].replace(r'\.+', '', regex=True)

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_102058/4266213702.py:2: SyntaxWarning: invalid escape sequence '\s'
  df['text'] = df['text'].str.replace('\s+', ' ', regex=True).str.strip()



### 9. Replace
* ['ሐ', 'ሑ', 'ሒ', 'ሓ', 'ሔ', 'ሖ'] with ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']
* ['ኀ', 'ኁ', 'ኂ', 'ኃ', 'ኄ', 'ኅ', 'ኆ'] with ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']
* ['ሠ', 'ሡ', 'ሢ', 'ሣ', 'ሤ', 'ሦ', 'ሦ', 'ሧ'] with ['ሰ, 'ሱ', 'ሲ', 'ሳ', 'ሴ', 'ስ', 'ሶ', 'ሷ']
* ['ዐ', 'ዑ', 'ዒ', 'ዓ', 'ዔ', 'ዕ', 'ዖ'] with ['አ', 'ኡ', 'ኢ', 'ኣ', 'ኤ', 'እ', 'ኦ']
* ['ጸ', 'ጹ', 'ጺ', 'ጻ', 'ጼ', 'ጽ', 'ጾ'] with ['ፀ', 'ፁ', 'ፂ', 'ፃ', 'ፄ', 'ፅ', 'ፆ']


In [28]:
letters = [
  [['ሐ', 'ሑ', 'ሒ', 'ሓ', 'ሔ', 'ሖ'], ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']],
  [['ኀ', 'ኁ', 'ኂ', 'ኃ', 'ኄ', 'ኅ', 'ኆ'], ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']],
  [['ሠ', 'ሡ', 'ሢ', 'ሣ', 'ሤ', 'ሦ', 'ሦ', 'ሧ'], ['ሰ', 'ሱ', 'ሲ', 'ሳ', 'ሴ', 'ስ', 'ሶ', 'ሷ']],
  [['ዐ', 'ዑ', 'ዒ', 'ዓ', 'ዔ', 'ዕ', 'ዖ'], ['አ', 'ኡ', 'ኢ', 'ኣ', 'ኤ', 'እ', 'ኦ']],
  [['ጸ', 'ጹ', 'ጺ', 'ጻ', 'ጼ', 'ጽ', 'ጾ'], ['ፀ', 'ፁ', 'ፂ', 'ፃ', 'ፄ', 'ፅ', 'ፆ']]
]

for letter in letters:
  for i in range(len(letter[0])):
    df['text'] = df['text'].str.replace(letter[0][i], letter[1][i])
    


### 10. Remove Non Amharic Characters 

In [30]:
# Clean English characters from the 'text' column
df['text'] = df['text'].str.replace(r'[A-Za-z]+', '', regex=True)